In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import sys
import os

sys.path.append('..')

from interpretDistill.fourierDistill import *
from interpretDistill.binaryTransformer import *

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

from ucimlrepo import fetch_ucirepo 
solar_flare = fetch_ucirepo(id=89) 
  
X = solar_flare.data.features 
y = solar_flare.data.targets['common flares']

X.columns = X.columns.str.replace(' ', '_')
y.name = 'common_flares'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
y.value_counts()

In [ ]:
bt_bit = BinaryTransformer(depth = 4, bit = True)
bt_bin = BinaryTransformer(depth = 4, bit = False)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
X_train_bit = bt_bit.fit_and_transform(X_train, y_train)
X_val_bit = bt_bit.transform(X_val)
X_test_bit = bt_bit.transform(X_test)

In [ ]:
X_train_bit.isnull().values.any()

In [ ]:
X_train_bin = bt_bin.fit_and_transform(X_train, y_train)
X_val_bin = bt_bin.transform(X_val)
X_test_bin = bt_bin.transform(X_test)

In [ ]:
X_train_bin.isnull().values.any()

In [ ]:
X_train_orig = pd.get_dummies(X_train, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})
X_val_orig = pd.get_dummies(X_val, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})
X_test_orig = pd.get_dummies(X_test, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})

In [ ]:
X_train_orig.isnull().values.any()

In [ ]:
print(f'dummy (orig) number of features: {X_train_orig.shape[1]}')
print(f'bit number of features: {X_train_bit.shape[1]}')
print(f'bin number of features: {X_train_bin.shape[1]}')

In [ ]:
X_train_orig = pd.get_dummies(X_train, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})
X_val_orig = pd.get_dummies(X_val, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})
X_test_orig = pd.get_dummies(X_test, columns=list(X.columns)).astype(int).replace({-1:-1, 0:-1, 1:1})

In [ ]:
rf_orig = RandomForestRegressor(max_depth = 6, criterion = 'absolute_error')
rf_bit = RandomForestRegressor(max_depth = 6, criterion = 'absolute_error')
rf_bin = RandomForestRegressor(max_depth = 6, criterion = 'absolute_error')

In [ ]:
# rf_orig.fit(X_train_orig, y_train)
# rf_bit.fit(X_train_bit, y_train)
# rf_bin.fit(X_train_bin, y_train)
rf_orig.fit(X_val_orig, y_val)
rf_bit.fit(X_val_bit, y_val)
rf_bin.fit(X_val_bin, y_val)

In [ ]:
print(f'[orig] train MSE: {mean_squared_error(rf_orig.predict(X_train_orig),y_train)}, val MSE: {mean_squared_error(rf_orig.predict(X_val_orig),y_val)}, test MSE: {mean_squared_error(rf_orig.predict(X_test_orig),y_test)}')
print(f'[bit] train MSE: {mean_squared_error(rf_bit.predict(X_train_bit),y_train)}, val MSE: {mean_squared_error(rf_bit.predict(X_val_bit),y_val)}, test MSE: {mean_squared_error(rf_bit.predict(X_test_bit),y_test)}')
print(f'[bin] train MSE: {mean_squared_error(rf_bin.predict(X_train_bin),y_train)}, val MSE: {mean_squared_error(rf_bin.predict(X_val_bin),y_val)}, test MSE: {mean_squared_error(rf_bin.predict(X_test_bin),y_test)}')

In [ ]:
print(f'[orig RF] train R2: {r2_score(rf_orig.predict(X_train_orig),y_train)}, val R2: {r2_score(rf_orig.predict(X_val_orig),y_val)}, test R2: {r2_score(rf_orig.predict(X_test_orig),y_test)}')
print(f'[bit RF] train R2: {r2_score(rf_bit.predict(X_train_bit),y_train)}, val R2: {r2_score(rf_bit.predict(X_val_bit),y_val)}, test R2: {r2_score(rf_bit.predict(X_test_bit),y_test)}')
print(f'[bin RF] train R2: {r2_score(rf_bin.predict(X_train_bin),y_train)}, val R2: {r2_score(rf_bin.predict(X_val_bin),y_val)}, test R2: {r2_score(rf_bin.predict(X_test_bin),y_test)}')

In [ ]:
ftd_orig = FTDistillCV(size_interactions = 3)
ftd_bit = FTDistillCV(size_interactions = 3)
ftd_bin = FTDistillCV(size_interactions = 3)

In [ ]:
# ftd_bit.fit(X_val_bit, rf_bit.predict(X_val_bit)) #rf_orig.predict(X_val))
# ftd_bin.fit(X_val_bin, rf_bin.predict(X_val_bin), bt_bin.no_interaction) #rf_orig.predict(X_val), bt_bin.no_interaction)
ftd_orig.fit(X_train_orig, rf_orig.predict(X_train_orig))
ftd_bit.fit(X_train_bit, rf_bit.predict(X_train_bit)) #rf_orig.predict(X_val))
ftd_bin.fit(X_train_bin, rf_bin.predict(X_train_bin), bt_bin.no_interaction) #rf_orig.predict(X_val), bt_bin.no_interaction)

In [ ]:
sum(ftd_orig.regression_model.coef_ != 0), sum(ftd_bit.regression_model.coef_ != 0), sum(ftd_bin.regression_model.coef_ != 0)

In [ ]:
print(f'[orig FTD, true y] train MSE: {mean_squared_error(ftd_orig.predict(X_train_orig),y_train)}, val MSE: {mean_squared_error(ftd_orig.predict(X_val_orig),y_val)}, test MSE: {mean_squared_error(ftd_orig.predict(X_test_orig),y_test)}')
print(f'[orig FTD, RF y] train MSE: {mean_squared_error(ftd_orig.predict(X_train_orig),rf_orig.predict(X_train_orig))}, val MSE: {mean_squared_error(ftd_orig.predict(X_val_orig),rf_orig.predict(X_val_orig))}, test MSE: {mean_squared_error(ftd_orig.predict(X_test_orig),rf_orig.predict(X_test_orig))}')
print(f'[bit FTD, true y] train MSE: {mean_squared_error(ftd_bit.predict(X_train_bit),y_train)}, val MSE: {mean_squared_error(ftd_bit.predict(X_val_bit),y_val)}, test MSE: {mean_squared_error(ftd_bit.predict(X_test_bit),y_test)}')
print(f'[bit FTD, RF y] train MSE: {mean_squared_error(ftd_bit.predict(X_train_bit),rf_bit.predict(X_train_bit))}, val MSE: {mean_squared_error(ftd_bit.predict(X_val_bit),rf_bit.predict(X_val_bit))}, test MSE: {mean_squared_error(ftd_bit.predict(X_test_bit),rf_bit.predict(X_test_bit))}')
print(f'[bin FTD, true y] train MSE: {mean_squared_error(ftd_bin.predict(X_train_bin),y_train)}, val MSE: {mean_squared_error(ftd_bin.predict(X_val_bin),y_val)}, test MSE: {mean_squared_error(ftd_bin.predict(X_test_bin),y_test)}')
print(f'[bin FTD, RF y] train MSE: {mean_squared_error(ftd_bin.predict(X_train_bin),rf_bin.predict(X_train_bin))}, val MSE: {mean_squared_error(ftd_bin.predict(X_val_bin),rf_bin.predict(X_val_bin))}, test MSE: {mean_squared_error(ftd_bin.predict(X_test_bin),rf_bin.predict(X_test_bin))}')

In [ ]:
print(f'[orig FTD, true y] train R2: {r2_score(ftd_orig.predict(X_train_orig),y_train)}, val R2: {r2_score(ftd_orig.predict(X_val_orig),y_val)}, test R2: {r2_score(ftd_orig.predict(X_test_orig),y_test)}')
print(f'[orig FTD, RF y] train R2: {r2_score(ftd_orig.predict(X_train_orig),rf_orig.predict(X_train_orig))}, val R2: {r2_score(ftd_orig.predict(X_val_orig),rf_orig.predict(X_val_orig))}, test R2: {r2_score(ftd_orig.predict(X_test_orig),rf_orig.predict(X_test_orig))}')
print(f'[bit RF, true y] train R2: {r2_score(ftd_bit.predict(X_train_bit),y_train)}, val R2: {r2_score(ftd_bit.predict(X_val_bit),y_val)}, test R2: {r2_score(ftd_bit.predict(X_test_bit),y_test)}')
print(f'[bit RF, RF y] train R2: {r2_score(ftd_bit.predict(X_train_bit),rf_bit.predict(X_train_bit))}, val R2: {r2_score(ftd_bit.predict(X_val_bit),rf_bit.predict(X_val_bit))}, test R2: {r2_score(ftd_bit.predict(X_test_bit),rf_bit.predict(X_test_bit))}')
print(f'[bin RF, true y] train R2: {r2_score(ftd_bin.predict(X_train_bin),y_train)}, val R2: {r2_score(ftd_bin.predict(X_val_bin),y_val)}, test R2: {r2_score(ftd_bin.predict(X_test_bin),y_test)}')
print(f'[bin RF, RF y] train R2: {r2_score(ftd_bin.predict(X_train_bin),rf_bin.predict(X_train_bin))}, val R2: {r2_score(ftd_bin.predict(X_val_bin),rf_bin.predict(X_val_bin))}, test R2: {r2_score(ftd_bin.predict(X_test_bin),rf_bin.predict(X_test_bin))}')

In [ ]:
# print(f'[bit RF, true y] train R2: {r2_score(ftd_bit.predict(X_train_bit),y_train)}, val R2: {r2_score(ftd_bit.predict(X_val_bit),y_val)}, test R2: {r2_score(ftd_bit.predict(X_test_bit),y_test)}')
# print(f'[bit RF, RF y] train R2: {r2_score(ftd_bit.predict(X_train_bit),rf_orig.predict(X_train))}, val R2: {r2_score(ftd_bit.predict(X_val_bit),rf_orig.predict(X_val))}, test R2: {r2_score(ftd_bit.predict(X_test_bit),rf_orig.predict(X_test))}')
# print(f'[bin RF, true y] train R2: {r2_score(ftd_bin.predict(X_train_bin),y_train)}, val R2: {r2_score(ftd_bin.predict(X_val_bin),y_val)}, test R2: {r2_score(ftd_bin.predict(X_test_bin),y_test)}')
# print(f'[bin RF, RF y] train R2: {r2_score(ftd_bin.predict(X_train_bin),rf_orig.predict(X_train))}, val R2: {r2_score(ftd_bin.predict(X_val_bin),rf_orig.predict(X_val))}, test R2: {r2_score(ftd_bin.predict(X_test_bin),rf_orig.predict(X_test))}')

In [ ]:
from itertools import compress

In [ ]:
sorted([i for i in zip(list(compress(ftd_orig.features, ftd_orig.regression_model.coef_ != 0)), list(compress(ftd_orig.regression_model.coef_, ftd_orig.regression_model.coef_ != 0)))], key = lambda x: abs(x[1]))[-3:]

In [ ]:
sorted([i for i in zip(list(compress(ftd_bit.features, ftd_bit.regression_model.coef_ != 0)), list(compress(ftd_bit.regression_model.coef_, ftd_bit.regression_model.coef_ != 0)))], key = lambda x: abs(x[1]))[-3:]

In [ ]:
sorted([i for i in zip(list(compress(ftd_bin.features, ftd_bin.regression_model.coef_ != 0)), list(compress(ftd_bin.regression_model.coef_, ftd_bin.regression_model.coef_ != 0)))], key = lambda x: abs(x[1]))[-3:]